In [ ]:
import os
import json

json_folder = 'GL_v1\\Labeled_GL_sample_1'
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

data_list = []
for file_name in json_files:
    file_path = os.path.join(json_folder, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        data_list.append(data)

chuyển thành bản đẹp


In [13]:
# Beautify all JSON files in the folder
for file_name in json_files:
    file_path = os.path.join(json_folder, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4, sort_keys=True)
print("All JSON files have been beautified.")

All JSON files have been beautified.


sửa các trường

In [ ]:
# Đồng bộ tên các trường from_name, to_name trong vuong.json theo duyet.json

vuong_path = os.path.join(json_folder, 'vuong.json')
duyet_path = os.path.join(json_folder, 'duyet.json')

with open(vuong_path, 'r', encoding='utf-8') as f:
    vuong_data = json.load(f)
with open(duyet_path, 'r', encoding='utf-8') as f:
    duyet_data = json.load(f)

# Lấy annotation đầu tiên có trường result trong duyet.json
duyet_ann = None
for item in duyet_data:
    if item.get('annotations'):
        for ann in item['annotations']:
            if 'result' in ann and ann['result']:
                duyet_ann = ann
                break
    if duyet_ann:
        break

if duyet_ann:
    duyet_fields = [
        (r.get('from_name'), r.get('to_name'))
        for r in duyet_ann['result']
    ]
    for item in vuong_data:
        if item.get('annotations'):
            for ann in item['annotations']:
                if 'result' in ann and ann['result']:
                    for i, r in enumerate(ann['result']):
                        if i < len(duyet_fields):
                            r['from_name'] = duyet_fields[i][0]
                            r['to_name'] = duyet_fields[i][1]
    with open(vuong_path, 'w', encoding='utf-8') as f:
        json.dump(vuong_data, f, ensure_ascii=False, indent=4, sort_keys=True)
    print("Đã đồng bộ tên các trường from_name, to_name trong vuong.json.")
else:
    print("Không tìm thấy annotation mẫu trong duyet.json.")

In [22]:
# Lập bảng cho từng trường from_name: mỗi trường là 1 bảng, các annotator là dòng, các choice (1-5) là cột, cell là tổng số lần chọn
# Đồng thời tính Fleiss' kappa cho từng trường

import pandas as pd
import numpy as np

def fleiss_kappa(M):
    """
    M: numpy array (n_items, n_categories), mỗi dòng là số annotator chọn mỗi choice cho 1 item
    """
    N, k = M.shape
    n_annotators = np.sum(M[0])
    P = np.sum(M, axis=0) / (N * n_annotators)
    P_i = (np.sum(M * M, axis=1) - n_annotators) / (n_annotators * (n_annotators - 1))
    P_bar = np.mean(P_i)
    P_e = np.sum(P * P)
    kappa = (P_bar - P_e) / (1 - P_e) if (1 - P_e) != 0 else np.nan
    return kappa

annotators = ['cuong', 'duyet', 'vuong']
choices = ['1', '2', '3', '4', '5']

# Lấy tất cả from_name có trong các file annotator
from_names = set()
for annotator in annotators:
    path = os.path.join(json_folder, f'{annotator}.json')
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    for item in data:
        anns = item.get('annotations', [])
        if not anns: continue
        ann = anns[0]
        for r in ann.get('result', []):
            fname = r.get('from_name')
            if fname:
                from_names.add(fname)
from_names = sorted(from_names)

for fname in from_names:
    # Tạo bảng tổng hợp annotator x choice
    rows = []
    for annotator in annotators:
        path = os.path.join(json_folder, f'{annotator}.json')
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        counts = {c: 0 for c in choices}
        for item in data:
            anns = item.get('annotations', [])
            if not anns: continue
            ann = anns[0]
            for r in ann.get('result', []):
                if r.get('from_name') == fname:
                    val = r.get('value', {}).get('choices', [])
                    if val and val[0] in choices:
                        counts[val[0]] += 1
        row = [counts[c] for c in choices]
        rows.append(row)
    df = pd.DataFrame(rows, columns=[f' {c}' for c in choices], index=annotators)
    df.index.name = 'Annotator'
    print(f'Bảng tổng hợp cho trường: {fname}')
    display(df)

    # Tạo ma trận Fleiss: mỗi dòng là 1 item, mỗi cột là số annotator chọn choice đó
    # rows2: annotator x item (choice value)
    rows2 = []
    n_items = None
    for annotator in annotators:
        path = os.path.join(json_folder, f'{annotator}.json')
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        item_choices = []
        for item in data:
            anns = item.get('annotations', [])
            if not anns: continue
            ann = anns[0]
            found = False
            for r in ann.get('result', []):
                if r.get('from_name') == fname:
                    val = r.get('value', {}).get('choices', [])
                    if val and val[0] in choices:
                        item_choices.append(val[0])
                        found = True
                        break
            if not found:
                item_choices.append(None)
        rows2.append(item_choices)
        if n_items is None:
            n_items = len(item_choices)
    M = np.zeros((n_items, len(choices)), dtype=int)
    for i in range(n_items):
        for j, annotator in enumerate(annotators):
            v = rows2[j][i]
            if v is not None:
                idx = choices.index(v)
                M[i, idx] += 1
    kappa = fleiss_kappa(M)
    print(f"Fleiss' kappa cho trường {fname}: {kappa:.4f}")

Bảng tổng hợp cho trường: All_helpfulness


,1,2,3,4,5
Annotator,,,,,
cuong,13,12,28,10,7
duyet,4,32,27,7,0
vuong,39,6,6,8,11


Fleiss' kappa cho trường All_helpfulness: 0.0679
Bảng tổng hợp cho trường: Comment_helpfulness


,1,2,3,4,5
Annotator,,,,,
cuong,8,5,11,13,33
duyet,5,24,27,13,1
vuong,6,9,34,13,8


Fleiss' kappa cho trường Comment_helpfulness: 0.0998
Bảng tổng hợp cho trường: Image_helpfulness


,1,2,3,4,5
Annotator,,,,,
cuong,46,2,3,7,12
duyet,44,7,11,8,0
vuong,42,3,3,12,10


Fleiss' kappa cho trường Image_helpfulness: 0.4714


In [ ]:
# Lập bảng tổng hợp annotator x choice và tính Fleiss' kappa cho từng trường from_name, xuất ra 1 bảng tổng hợp kết quả kappa

import pandas as pd
import numpy as np

def fleiss_kappa(M):
    """
    M: numpy array (n_items, n_categories), mỗi dòng là số annotator chọn mỗi choice cho 1 item
    """
    N, k = M.shape
    n_annotators = np.sum(M[0])
    P = np.sum(M, axis=0) / (N * n_annotators)
    P_i = (np.sum(M * M, axis=1) - n_annotators) / (n_annotators * (n_annotators - 1))
    P_bar = np.mean(P_i)
    P_e = np.sum(P * P)
    kappa = (P_bar - P_e) / (1 - P_e) if (1 - P_e) != 0 else np.nan
    return kappa

annotators = ['cuong', 'duyet', 'vuong']
choices = ['1', '2', '3', '4', '5']

# Lấy tất cả from_name có trong các file annotator
from_names = set()
for annotator in annotators:
    path = os.path.join(json_folder, f'{annotator}.json')
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    for item in data:
        anns = item.get('annotations', [])
        if not anns: continue
        ann = anns[0]
        for r in ann.get('result', []):
            fname = r.get('from_name')
            if fname:
                from_names.add(fname)
from_names = sorted(from_names)

kappa_results = {}
for fname in from_names:
    # Tạo bảng tổng hợp annotator x choice
    rows = []
    for annotator in annotators:
        path = os.path.join(json_folder, f'{annotator}.json')
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        counts = {c: 0 for c in choices}
        for item in data:
            anns = item.get('annotations', [])
            if not anns: continue
            ann = anns[0]
            for r in ann.get('result', []):
                if r.get('from_name') == fname:
                    val = r.get('value', {}).get('choices', [])
                    if val and val[0] in choices:
                        counts[val[0]] += 1
        row = [counts[c] for c in choices]
        rows.append(row)
    df = pd.DataFrame(rows, columns=[f' {c}' for c in choices], index=annotators)
    df.index.name = 'Annotator'
    print(f'Bảng tổng hợp cho trường: {fname}')
    display(df)

    # Tạo ma trận Fleiss: mỗi dòng là 1 item, mỗi cột là số annotator chọn choice đó
    rows2 = []
    n_items = None
    for annotator in annotators:
        path = os.path.join(json_folder, f'{annotator}.json')
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        item_choices = []
        for item in data:
            anns = item.get('annotations', [])
            if not anns: continue
            ann = anns[0]
            found = False
            for r in ann.get('result', []):
                if r.get('from_name') == fname:
                    val = r.get('value', {}).get('choices', [])
                    if val and val[0] in choices:
                        item_choices.append(val[0])
                        found = True
                        break
            if not found:
                item_choices.append(None)
        rows2.append(item_choices)
        if n_items is None:
            n_items = len(item_choices)
    M = np.zeros((n_items, len(choices)), dtype=int)
    for i in range(n_items):
        for j, annotator in enumerate(annotators):
            v = rows2[j][i]
            if v is not None:
                idx = choices.index(v)
                M[i, idx] += 1
    kappa = fleiss_kappa(M)
    kappa_results[fname] = kappa
    print(f"Fleiss' kappa cho trường {fname}: {kappa:.4f}")

# Xuất bảng tổng hợp Fleiss' kappa
kappa_df = pd.DataFrame.from_dict(kappa_results, orient='index', columns=['Fleiss\' kappa'])
display(kappa_df)